In [6]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
)

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors. Also, include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


directors, movies
Invoking: `sql_db_schema` with `movies`



CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800	In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but 	Enter the World of Pandora.	19995	4762
43598	Pirates of the Caribbean: At World's End	300000000	139	2007-05-19	961000000	Pirates of the Caribbean: At World's End	6.9	4500	Captain Barbossa, long believed to be dead, has come back to

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors. Also, include their gross revenue.',
 'output': 'The movies that have the highest votes but the lowest budgets are:\n\n1. Movie: Inception\n   - Director ID: 4765\n   - Budget: $160,000,000\n   - Gross Revenue: $825,532,764\n\n2. Movie: The Dark Knight\n   - Director ID: 4765\n   - Budget: $185,000,000\n   - Gross Revenue: $1,004,558,444\n\n3. Movie: Avatar\n   - Director ID: 4762\n   - Budget: $237,000,000\n   - Gross Revenue: $2,787,965,087\n\n4. Movie: The Avengers\n   - Director ID: 4769\n   - Budget: $220,000,000\n   - Gross Revenue: $1,519,557,910\n\n5. Movie: Deadpool\n   - Director ID: 5151\n   - Budget: $58,000,000\n   - Gross Revenue: $783,112,979\n\n6. Movie: Interstellar\n   - Director ID: 4765\n   - Budget: $165,000,000\n   - Gross Revenue: $675,120,017\n\n7. Movie: Django Unchained\n   - Director ID: 4927\n   - Budget: $100,000,000\n   - Gross Revenue: